In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\siddh\\OneDrive\\Desktop\\PERSONAL\\College\\Text-Summarizer-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int  # ADD THIS
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int
    fp16: bool  # ADD THIS
    report_to: list  # ADD THIS

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,  # ADD THIS
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,  # FIX: was params.evaluation_strategy
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            fp16 = params.fp16,  # ADD THIS
            report_to = params.report_to  # ADD THIS
        )

        return model_trainer_config

In [7]:
import os
import torch
import gc
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Clear GPU memory and check availability
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print(f"GPU available: {torch.cuda.get_device_name(0)}")
            print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
        else:
            print("GPU not available, using CPU (will be slower)")
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")
        
        # Load tokenizer and model with optimizations
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_t5 = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_ckpt,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        ).to(device)
        
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)
        
        # Loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # Re-tokenize with T5 tokenizer to fix vocabulary mismatch
        def retokenize_function(examples):
            # Add T5 task prefix
            inputs = ["summarize: " + doc for doc in examples["dialogue"]]
            model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=False)
            
            # Tokenize targets
            with tokenizer.as_target_tokenizer():
                labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding=False)
            
            model_inputs["labels"] = labels["input_ids"]
            return model_inputs
        
        # Re-tokenize the dataset and remove problematic columns
        print("Re-tokenizing dataset with T5 tokenizer...")
        dataset_samsum_pt = dataset_samsum_pt.map(
            retokenize_function, 
            batched=True,
            remove_columns=dataset_samsum_pt["train"].column_names  # Remove all original columns
        )
        
        # Use smaller dataset for faster training (remove these lines for full dataset)
        print("Using subset of data for faster training...")
        train_dataset = dataset_samsum_pt["train"].select(range(2000))  # Use 2000 samples instead of ~14k
        eval_dataset = dataset_samsum_pt["validation"].select(range(400))  # Use 400 samples instead of ~800
        
        print(f"Training on {len(train_dataset)} samples")
        print(f"Evaluating on {len(eval_dataset)} samples")
        
        # Optimized training arguments
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,  # Reduced from 3 to 1 for testing
            warmup_steps=50,     # Reduced from 500
            per_device_train_batch_size=4,  # Increased from 2
            per_device_eval_batch_size=4,   # Increased from 2
            weight_decay=0.01,
            logging_steps=10,
            eval_strategy='steps',
            eval_steps=50,       # Reduced from 500
            save_steps=100,      # Reduced from 1000
            gradient_accumulation_steps=2,  # Reduced from 8
            fp16=True,           # Enable mixed precision for speed
            dataloader_pin_memory=True,     # Enable for GPU
            dataloader_num_workers=0,       # Set to 0 to avoid multiprocessing issues
            max_steps=500,       # Limit total steps for testing
            save_total_limit=2,  # Keep only 2 checkpoints
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            report_to=[],        # Disable wandb
            remove_unused_columns=True,  # Remove unused columns automatically
        )
        
        # Create trainer
        trainer = Trainer(
            model=model_t5, 
            args=trainer_args,
            tokenizer=tokenizer, 
            data_collator=seq2seq_data_collator,
            train_dataset=train_dataset,  # Use smaller dataset
            eval_dataset=eval_dataset     # Use smaller dataset
        )
        
        print("Starting training...")
        trainer.train()
        
        print("Training completed! Saving model...")
        
        # Save model and tokenizer
        model_t5.save_pretrained(os.path.join(self.config.root_dir, "t5-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))
        
        print(f"Model saved to: {os.path.join(self.config.root_dir, 't5-samsum-model')}")
        print(f"Tokenizer saved to: {os.path.join(self.config.root_dir, 'tokenizer')}")
        
        # Clear GPU memory after training
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

c:\Users\siddh\OneDrive\Desktop\PERSONAL\College\Text-Summarizer-Project\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-11 23:11:09,451] INFO: config: PyTorch version 2.6.0 available.]


In [8]:
# Run training
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    # Clean up memory on error
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    raise e

[2025-06-11 23:11:10,162] INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-11 23:11:10,170] INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-11 23:11:10,172] INFO: common: created directory at: artifacts]
[2025-06-11 23:11:10,174] INFO: common: created directory at: artifacts/model_trainer]
GPU not available, using CPU (will be slower)
Using device: cpu
Re-tokenizing dataset with T5 tokenizer...


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\siddh\OneDrive\Desktop\PERSONAL\College\Text-Summarizer-Project\textS\lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 4407.31 examples/s]
C:\Users\siddh\AppData\Local\Temp\ipykernel_25300\1462893022.py:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Using subset of data for faster training...
Training on 2000 samples
Evaluating on 400 samples
Starting training...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,2.564300,2.196485
100,2.195500,2.015077
150,2.327900,1.964090
200,2.206000,1.938570
250,2.237500,1.923598
300,2.159300,1.914429
350,2.064800,1.908763
400,2.053000,1.907045
450,2.168800,1.901887
500,2.084800,1.900143


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Training completed! Saving model...
Model saved to: artifacts/model_trainer\t5-samsum-model
Tokenizer saved to: artifacts/model_trainer\tokenizer
